In [34]:
# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제프로젝트  AI hub 기타등등..
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [35]:
# 텍스트 전처리
# unicode 정규화, 특수문자처리
# NFD Normalization Form Decomposition -> 분해가능한 모든 문자를 분해한다

import unicodedata
unicodedata.normalize('NFD',"e'")

"e'"

$\acute{e}$   

단일문자 U+00E9

$e$ + $\acute{}$

$e$ + 악센트

###### 같은 문장처럼 보이지만 내부적으로 다른 바이트조합을 갖는 경우 -> 동일한 내부 표현으로 바꿔주는 과정

In [36]:
# 2. 전처리 함수
def preprocess_stentence(sentence, is_korean = False):
    '''
    Unicode정규화, 특수문자처리
    Args : 
        sentence: 원본
        is_korean : 한국어 여부
    return:
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD',sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip()
    # 정규식으로 특수문자 전후에 공백 추가
    # 안녕하세요! -> 안녕하세요 !
    # r"([?.!,])"   문자중에 ?.!, 나오면 문자를 그룹으로 캡쳐
    # r" \1 " 캡처한 문자( \1 ) 앞뒤로 공백을 하나씩 넣는다
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

In [37]:
korean_processed = [preprocess_stentence(kor) for kor in korean_sentences]
english_processed = [preprocess_stentence(eng) for eng in english_sentences]

In [38]:
# 문장을 정수 시퀀스로 변환
# 3. 토크나이져
def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary구축, word to index mapping
    Args:
        sentence : 문장 리스트
    Returns:
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', # 필터 비활성화
        oov_token='<unk>', # out of vocabulary 
    )
    # 모든 문장으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer


In [39]:
# 한국어, 영어 각가의 토크나이져 생성
korean_tokenizer = create_tokenizer(korean_processed)
english_tokenizer = create_tokenizer(english_processed)

# 단어 사전 
korean_vocab_size = len(korean_tokenizer.word_index) + 1
english_vocab_size = len(english_tokenizer.word_index) + 1
print(f'한국어 사전 크기 : {korean_vocab_size}')
print(f'영어 사전 크기 : {english_vocab_size}')

한국어 사전 크기 : 25
영어 사전 크기 : 30


In [40]:
# 4. 정수시퀀스 변경
def encode_sentences(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일 길이로 맞추는
    Args: 
        tokenizer 
        sentences : 문장리스트
        maxlen : 가장 긴 문장길이
    Returns:
        패딩된 정수 시퀀스배열  tf의 pad_sequence
    '''
    sequences = tokenizer.texts_to_sequences(sentences) # [i love you] [1, 5, 7]
    # 길이 맞추기 (패딩추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequences,
        maxlen=maxlen,
        padding='post'
    )
    return padded

- 딥러닝 RNN/LSTM/GUR  전통적인 시계열 처리 기법
    - pre
    - RNN 앞에서부터 읽어 -> 실제 단어가 뒤쪽에 몰려있다
- Transformer 계열은 post
    - 위치정보 -> 앞 뒤 순서를 그대로 유지해야 자연스럽다 -> 뒤쪽 패딩은.. masking 처리

In [41]:
# 최대 시퀀스의 길이 결정(가장 긴 문장기준)
max_korean_len = max(len(s.split()) for s in korean_processed)
max_english_len = max(len(s.split()) for s in english_processed)
print(f'시퀀스의 최대길이')
print(f'한국어: {max_korean_len}')
print(f'영어: {max_english_len}')
# 인코딩 수행
korean_tensor = encode_sentences(korean_tokenizer, korean_processed, max_korean_len)
english_tensor = encode_sentences(english_tokenizer, english_processed, max_english_len)
print(f'인코딩 첫번째 문장')
print(f'한국어 인코딩 문장: {korean_tensor[0]}')
print(f'영어 인코딩 문장: {english_tensor[0]}')
print(f'한국어 문장: {korean_processed[0]}')
print(f'영어 문장: {english_processed[0]}')

BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

# tensorflow Dataset 객체
# 배치처리 와 셔플
dataset = tf.data.Dataset.from_tensor_slices( (korean_tensor, english_tensor) )
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(f'데이터셋 준비 완료')
print(f'전체 샘플 수 : {len(korean_tensor)}')
print(f'배치 크기 : {BATCH_SIZE}')
print(f'배치 수 : {len(korean_tensor) // BATCH_SIZE}')

시퀀스의 최대길이
한국어: 3
영어: 5
인코딩 첫번째 문장
한국어 인코딩 문장: [3 0 0]
영어 인코딩 문장: [7 0 0 0 0]
한국어 문장: <start>안녕하세요<end>
영어 문장: <start>hello<end>
데이터셋 준비 완료
전체 샘플 수 : 10
배치 크기 : 2
배치 수 : 5


-- step2 --

seq2seq 구조
```
Encoder
    입력문장 -> Embedding -> LSTM + Hidden States
Decoder
    <start>토큰 -> Embedding -> LSTM + context -> 단어 예측
    예측단어 -> 다음입력 -> 반복(<end> 나올때 까지)
```

In [ ]:
# Encoder 클래스

class Encoder(tf.keras.Model):
    '''
    입력문장을 고차원 벡터로 압축
    Embedding -> LSTM -> Hidden States 출력
    구조 :
        입력(정수시퀀스) -> Embedding -> LSTM -> 모든 타입스텝의 출력
    '''
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        '''
        Args:
            vocab_size : 단어사전 크기(임베딩 테이블 크기)
            embedding_dim : 임베딩 벡터 차원
            enc_units : LMS 유닛수(hidden_state 차원)
            batch_size : 배치크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        # Embedding Layer : 정수 --> 밀집벡터(Dense Vector) : 작은차원 대부분의 값이 0이 아님, 연속된 실수값 공간효율성
        # 학습과정을 통해 단어 간 의미적 관계학습 -> 유사 단어는 가까운 벡터 공간에 위치
        # 단점 : 훈련필요(사전학습 또는 임베딩을 학습)

        # 희소벡터 : 대부분의 값이 0(대표적인 원-핫벡터) 차원이 크다, 단어간 충돌이없다 단어간 유사도 표현못함
        # 희소벡터 : 규칙기반(원핫 Vow) - 초창기 자연어모델
        # 밀집벡터 : 학습기반(Word2Vec Glove Embedding)
        
        # mask_zero = True  패딩에 마스크처리를 해서 모델이 해석하지 않게 한다
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, embedding_dim, mask_zero=True
        )
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequence = True, #[batch, seq_len, units]
            return_state=True, #(output, h, c)
            recurrent_initializer = 'glorot_uniform', # 가중치 초기화    
        )
    def call(self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args:
            x : 입력 시퀀스 [ baatch_size, seq_len ]
            hidden : 초기 hidden state(첫 호출시 0벡터)
        Returns:
            output: 모든 타임스탬프의 출력 [batch, seq_len, enc_units]
            state_h: 마지막 hidden state [batch, enc_units]
            state_c: 마지막 cell state [batch, enc_units]
        '''

In [43]:
class ABC():
    def __call__(self, data):
        self.call(data)
    
    def call(self, data):
        print('호출', data)

a=ABC()
a(100)

호출 100
